# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J DB 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

True

---

## 2. Neo4j 기반 질의 응답 시스템 (GraphRAG) 구현하기

- Graph DB를 기반으로 한 질의 응답 시스템(GraphRAG)은 전통적인 벡터 기반 RAG 시스템보다 더 정확하고 연관성 있는 답변을 제공할 수 있습니다. 
- Graph RAG는 "연결된 관계"를 따라가며 정보를 확장합니다.

- **특장점**:

   - **정확한 관계 검색**: 그래프 데이터베이스의 관계 중심 구조를 활용해 복잡한 연결 패턴을 찾을 수 있습니다
   - **컨텍스트 유지**: 엔티티 간의 관계를 유지하여 더 풍부한 컨텍스트를 제공합니다
   - **구조화된 정보 검색**: 단순 텍스트 검색이 아닌 구조화된 방식으로 정보를 검색합니다

- **필요 사항**:
   - Neo4j 데이터베이스 (영화 데이터 포함)
   - 필요 패키지: `langchain-neo4j`, `langchain-openai`

## 1) Graph DB 초기화 및 테스트

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE"),
    enhanced_schema=True,   # 향상된 스키마 사용 설정
)

- Cypher 쿼리

In [3]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

## 2) Graph Vector DB 초기화 및 테스트

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

# Neo4j 데이터베이스에 이미 생성된 벡터 인덱스에 연결하는 Neo4jVector 인스턴스 생성
graph_vector = Neo4jVector.from_existing_index(
    embeddings,  # 사용할 임베딩 모델 지정
    url=os.getenv("NEO4J_URI"),  # Neo4j 데이터베이스 연결 URI (환경 변수에서 가져옴)
    username=os.getenv("NEO4J_USERNAME"),  # Neo4j 데이터베이스 사용자 이름
    password=os.getenv("NEO4J_PASSWORD"),  # Neo4j 데이터베이스 비밀번호
    index_name="movie_content_embeddings",  # 사용할 벡터 인덱스 이름 (이미 Neo4j에 생성되어 있어야 함)
    text_node_property="overview",  # 텍스트 검색 시 반환할 노드의 속성 (영화 개요)
)

- 벡터 검색 (유사도 기준)

In [5]:
# 한국어로 된 자연어 쿼리를 사용하여 의미적으로 유사한 영화 검색
query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."

# 유사도 검색 수행
similar_docs = graph_vector.similarity_search_with_score(
    query,
    k=5, # 유사도 상위 5개 문서 검색
    return_embeddings=False, # 임베딩 반환 안함 (결과 간소화)
)

# 각 문서와 해당 유사도 점수를 함께 표시
for doc, score in similar_docs:
    print(f"줄거리: {doc.page_content[:100]}..., 유사도: {score}")
    print("영화 제목:", doc.metadata.get("title"))
    print("-" * 50)  

줄거리: The relationship between Sergeant Stryker and a group of rebellious recruits is made difficult by th..., 유사도: 0.7102100849151611
영화 제목: Sands of Iwo Jima
--------------------------------------------------
줄거리: The true story of how businessman Oskar Schindler saved over a thousand Jewish lives from the Nazis ..., 유사도: 0.7087187767028809
영화 제목: Schindler's List
--------------------------------------------------
줄거리: A US Fighter pilot's epic struggle of survival after being shot down on a mission over Laos during t..., 유사도: 0.7083426117897034
영화 제목: Rescue Dawn
--------------------------------------------------
줄거리: Wounded in Africa during World War II, Nazi Col. Claus von Stauffenberg returns to his native German..., 유사도: 0.7021337747573853
영화 제목: Valkyrie
--------------------------------------------------
줄거리: In a place where killers are celebrated as heroes, these filmmakers challenge unrepentant death-squa..., 유사도: 0.7017200589179993
영화 제목: The Act of Killing
----------------

----

# 3. 벡터-그래프 하이브리드 RAG
- Vector Graph Hybrid RAG : 구조적 하이브리드
    - 검색의 시작은 벡터(비정형)
    - 확장은 그래프(정형)
- 단편적인 정보 검색을 넘어 데이터 간의 관계와 연결성을 활용하기 위함

[핵심 프로세스]
- Entry Point (진입점) 찾기
    - 사용자의 질문과 가장 유사한 노드를 벡터 검색으로 빠르게 찾아냅니다.
- Knowledge Expansion (지식 확장)
    - 찾아낸 노드를 기점으로 관계(:ACTED_IN)를 따라가며 인간의 기억 모델처럼 정보를 확장합니다.
- Context Enrichment (문맥 풍부화)
    - 단순 검색 결과뿐만 아니라 관련 배우의 필모그래피까지 포함하여 LLM에게 풍부한 '배경지식'을 제공합니다.

### 1) 초기 설정 및 DB 연결

In [6]:
import os
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph, Neo4jVector
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 환경 변수 로드
load_dotenv(override=True)

# Neo4j 연결 (Cypher용)
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE"),
    enhanced_schema=True
)

# 모델 및 DB 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Neo4j 벡터 저장소 연결 (검색용)
vector_store = Neo4jVector.from_existing_index(
    embeddings,
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    index_name="movie_content_embeddings",
    text_node_property="overview",
)

### 2) 데이터 조회 함수
- 복잡한 Cypher 쿼리를 실행하고 데이터를 가져오는 역할을 수행함.

In [7]:
def get_movie_details_and_actors(movie_titles):
    """영화 제목 목록을 받아 상세 정보와 출연진을 조회합니다."""
    query = """
    MATCH (m:Movie)
    WHERE ANY(t IN $titles WHERE m.title CONTAINS t)
    OPTIONAL MATCH (m)<-[:ACTED_IN]-(a:Person)
    RETURN 
        m.title as title, 
        m.released as released, 
        m.rating as rating, 
        m.overview as overview,
        collect(a.name) as actor_names,
        collect(elementId(a)) as actor_ids
    """
    return graph.query(query, params={"titles": movie_titles})

In [8]:
def get_actor_filmography(actor_ids, exclude_titles):
    """배우들의 ID를 받아 다른 출연작들을 조회합니다."""
    if not actor_ids:
        return []
        
    query = """
    MATCH (a:Person)
    WHERE elementId(a) IN $actor_ids
    MATCH (a)-[:ACTED_IN]->(m:Movie)
    WHERE NOT m.title IN $exclude_titles   // 전달 받은 영화는 제외
    RETURN 
        a.name as actor_name, 
        collect({title: m.title, released: m.released}) as other_movies
    """
    return graph.query(query, params={"actor_ids": actor_ids, "exclude_titles": exclude_titles})

### 3) 데이터 포맷팅 함수 (Context Preparation)

In [9]:
def format_context_for_llm(movies, filmographies):
    """조회된 데이터를 하나의 문자열 문맥으로 합칩니다."""
    context_parts = ["## 검색된 영화 정보"]
    
    for m in movies:
        actors = ", ".join(m['actor_names']) if m['actor_names'] else "정보 없음"
        context_parts.append(
            f"- 영화: {m['title']} ({m['released']})\n"
            f"  평점: {m['rating']}\n"
            f"  배우: {actors}\n"
            f"  줄거리: {m['overview'][:100]}..."
        )
    
    if filmographies:
        context_parts.append("\n## 출연 배우의 다른 작품들")
        for f in filmographies:
            # 최대 3개까지만 표시 (너무 길어지지 않게)
            titles = [f"{m['title']}({m['released']})" for m in f['other_movies'][:3]]
            context_parts.append(f"- {f['actor_name']}: {', '.join(titles)}")
            
    return "\n".join(context_parts)

### 4) 메인 RAG 로직 함수 (Orchestration)
- 벡터 검색 -> 영화 제목 찾기 -> 상세 정보 조회(배우 포함) -> 필모그래피 조회(배우의 다른 작품) -> 포맷팅

In [10]:
def movie_graph_search_orchestrator(user_query):
    """벡터 검색 -> 상세 정보 조회 -> 필모그래피 조회 -> 포맷팅 과정을 총괄합니다."""
    
    # 1. 유사한 영화 제목 찾기 (Vector Search)
    docs = vector_store.similarity_search(user_query, k=3)
    found_titles = [doc.metadata.get("title") for doc in docs if doc.metadata.get("title")]
    
    if not found_titles:
        return "관련 정보를 찾을 수 없습니다."
    
    # 2. 영화 상세 정보 및 배우 ID 가져오기 (Graph Search 1)
    movie_data = get_movie_details_and_actors(found_titles)
    

    # 3. 모든 배우 ID 수집 (중복 제거)
    all_actor_ids = []
    for m in movie_data:
        all_actor_ids.extend(m['actor_ids'])
    all_actor_ids = list(set(all_actor_ids))  # 중복 제거
    
    
    # 4. 배우들의 다른 작품 가져오기 (Graph Search 2)
    film_data = get_actor_filmography(all_actor_ids, found_titles)
    
    
    # 5. 최종 텍스트 생성
    return format_context_for_llm(movie_data, film_data)

### 5) 증강, 생성 : LLM을 붙여서 최종 RAG 완성


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

def main_chain(query: str) -> str:
    # LLM 객체 생성
    llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)

    # Prompt 템플릿 정의
    template = '''당신은 영화 추천 전문가로서 오직 주어진 정보에 기반하여 객관적이고 정확한 답변을 제공합니다.

    [주어진 영화 정보]
    {context}

    [질문]
    {question}

    # 답변 작성 지침:
    1. 제공된 영화 정보에 명시된 사실만 사용하세요.
    2. 간결하고 정확하게 답변하세요.
    3. 제공된 정보에 없는 내용은 "제공된 정보에서 해당 내용을 찾을 수 없습니다"라고 답하세요.
    4. 영화의 제목, 평점 등 주요 정보를 포함해서 답변하세요.
    5. 한국어로 자연스럽고 이해하기 쉽게 답변하세요.
    '''

    # Prompt 객체 생성
    prompt = ChatPromptTemplate.from_template(template)

    # RAG 체인 구성(딕셔너리를 체인에 넣으면 자동으로 **RunnableParallel**로 변환됨)
    rag_input = {
        "context": RunnableLambda(movie_graph_search_orchestrator),   # vector 질문 → 그래프 검색 결과
        "question": RunnablePassthrough()    # 질문 그대로 전달
    }

    graph_rag_chain = rag_input| prompt | llm | StrOutputParser()

    return graph_rag_chain.invoke(query)

In [12]:
query = "2차 세계대전 영화 중에서 평점이 높은 작품과 그 배우들의 다른 작품을 알려줘"
answer = main_chain(query)

In [13]:
print(answer)

2차 세계대전 영화 중 평점이 높은 작품은 다음과 같습니다.

- 영화: The Thin Red Line (1998-12-25)  
  평점: 7.2  
  주요 배우 및 다른 작품:  
  - Sean Penn: Fast Times at Ridgemont High(1982-08-13), Shanghai Surprise(1986-08-29), We're No Angels(1989-12-15)  
  - George Clooney: From Dusk Till Dawn(1996-01-19), Batman & Robin(1997-06-20), The Peacemaker(1997-09-26)  
  - Adrien Brody: The Last Time I Committed Suicide(1997-06-20), Summer of Sam(1999-07-02), The Pianist(2002-09-24)  
  - Jim Caviezel: Frequency(2000-04-28), Pay It Forward(2000-10-12), Madison(2001-01-23)  
  - Ben Chaplin: Lost Souls(2000-10-13), Birthday Girl(2001-09-06), Murder by Numbers(2002-04-19)  

또 다른 2차 세계대전 영화인 The Good German(평점 5.9)도 있으나 평점이 더 낮습니다.

제공된 정보에서 이 외에 다른 2차 세계대전 영화는 찾을 수 없습니다.


- 질문 2

In [14]:
query = "코메디, 로맨스 영화중에 평점이 높은 작품과 그 배우들의 다른 작품을 알려줘"
answer = main_chain(query)

In [15]:
print(answer)

코메디, 로맨스 영화 중 평점이 가장 높은 작품은 1990년 개봉한 **Metropolitan**으로 평점은 7.0입니다. 주요 배우는 Edward Clements, Chris Eigeman, Taylor Nichols, Carolyn Farina, Isabel Gillies입니다. 다만, 이 배우들의 다른 작품 정보는 제공된 정보에서 찾을 수 없습니다.
